<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part4_%EC%A0%9C3%ED%9A%8C_%EA%B8%B0%EC%B6%9C%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#제1유형
문제 1 : 결측치가 있는 데이터의 행을 제거하고, 앞에서부터 70% 데이터만 활용해 'f1' 컬럼 1사분위 값을 구하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('members.csv')
df = df.dropna(axis=0)

l = int(len(df) * 0.7)
df = df.iloc[:l, ]

print(int(df['f1'].quantile(0.25)))

57


문제 2 : 2000년 데이터 중 2000년 평균보다 큰 값의 데이터 수를 구하시오 (index : 연도)

In [ ]:
import pandas as pd
df = pd.read_csv('year.csv', index_col = 'Unnamed: 0')
df = df.loc[2000, :]

m = df.mean()
(df>m).sum()

np.int64(100)

문제3 : 결측치가 가장 많은 칼럼명을 출력하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('members.csv')
df.isnull().sum().sort_values(ascending=False).index[0]

#제2유형
여행 보험 패키지 상품을 구매할 확률값을 예측하시오

In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.rename(columns={'Unnamed: 0' : 'ID'}, inplace=True)
train.head(3)

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,888,28,Private Sector/Self Employed,Yes,1250000,6,1,No,No,0
1,1308,31,Private Sector/Self Employed,Yes,1250000,7,1,No,No,0
2,151,29,Private Sector/Self Employed,Yes,1200000,7,0,No,No,1


데이터 전처리

In [ ]:
#0. Pop
target = train.pop('TravelInsurance')

#1. Unnamed : 0 삭제
train = train.drop('Unnamed: 0', axis=1)
test = test.drop('Unnamed: 0', axis=1)


In [ ]:
#2. 범주형 LE
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data = pd.concat([train, test], axis=0)

cols = data.select_dtypes(include='O').columns
for c in cols:
  data[c] = le.fit_transform(data[c])

train = data.iloc[:len(train)]
test = data.iloc[len(train):]

In [ ]:
#3. 연속형 Scaler
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

train['AnnualIncome'] = scaler.fit_transform(train[['AnnualIncome']])
test['AnnualIncome'] = scaler.transform(test[['AnnualIncome']])

/tmp/ipython-input-3605567613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['AnnualIncome'] = scaler.fit_transform(train[['AnnualIncome']])
/tmp/ipython-input-3605567613.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['AnnualIncome'] = scaler.transform(test[['AnnualIncome']])


모델 학습

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state = 99)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape )

(1043, 8) (447, 8) (1043,) (447,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

def base_model_selection(model, name):
  model.fit(X_train, y_train)
  pred = model.predict_proba(X_val)
  score = roc_auc_score(y_val, pred[:,1])
  print(name, ' score : ', score)


model = RandomForestClassifier(random_state = 99)
base_model_selection(model, 'rfr')

model = LogisticRegression(random_state = 99)
base_model_selection(model, 'lr')

model = DecisionTreeClassifier(random_state = 99)
base_model_selection(model, 'dt')

model = xgb.XGBClassifier(random_state = 99)
base_model_selection(model, 'xgb')

model = lgb.LGBMClassifier(random_state = 99)
base_model_selection(model, 'lgb')

#->lgb

rfr  score :  0.7961167227833894
lr  score :  0.7450953984287318
dt  score :  0.7255443322109988
xgb  score :  0.8122558922558922
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 375, number of negative: 668
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points in the train set: 1043, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.359540 -> initscore=-0.577362
[LightGBM] [Info] Start training from score -0.577362
lgb  score :  0.8144556677890011


In [ ]:
#0.               범주형 LE, MinMax - lightgbm - 0.8031200897867563
#1. Unnamed 삭제, 범주형 LE, MinMax - lightgbm - 0.8144556677890011
#2. Unnamed 삭제, 범주형 LE, Robust - lightgbm - 0.8144556677890011
#3. Unnamed 삭제, 범주형 OH, Robust - lightgbm - 0.8144556677890011

파라미터 수정

In [ ]:
from sklearn.model_selection import GridSearchCV

model = lgb.LGBMClassifier(random_state = 99)

param_grid = {
    'n_estimators' : [50, 100, 150, 200],
    'learning_rate' : [0.001, 0.01, 0.05, 0.1, 0.5, 1],
}

grid_search = GridSearchCV(
    estimator = model,
    param_grid = param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 375, number of negative: 668
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points in the train set: 1043, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.359540 -> initscore=-0.577362
[LightGBM] [Info] Start training from score -0.577362
{'learning_rate': 0.01, 'n_estimators': 100}
0.2672169977181761


저장

In [ ]:
model = grid_search.best_estimator_
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

np.float64(0.8045903479236813)

In [ ]:
pred = model.predict_proba(test)

res = pd.DataFrame({
    'index' : test.index,
    'y_pred' : pred[:, 1]
})
res.to_csv('result.csv', index=False)

In [ ]:
saved = pd.read_csv('result.csv')
saved

,index,y_pred
0,0,0.2922044865
1,1,0.3246214569
2,2,0.1872144230
3,3,0.7554344049
4,4,0.1622647318
5,5,0.7589660952
6,6,0.2806263604
7,7,0.2202629119
8,8,0.6232455058
9,9,0.2068138369


In [ ]:
res

,index,y_pred
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
5,5,1
6,6,0
7,7,0
8,8,1
9,9,0
